### 레퍼런스 음성 정보가 엠베딩으로 저장된 .npz를 파일을 불러와 텍스트->음성파일로 바꿈
1. embedding_path: 조정하여 원하는 엠베딩 파일로 바꿀 수 있음, 'conditioning_latents.npz'는 피노키오 아저씨, npz 저장 코드는 숮이가 갖고있음
2. output_folder: 음성 저장할 파일을 생성해줘야함(최종 음성파일이 저장되는 곳,tortoise/result/###)
3. pcm_audio: preset 속성으로 생성 속도(= 음성 퀄러티와 반비례) 조정가능
    - ultra_fast: 빠르게 생성되나 음성 품질 안좋음(배경음, 끊김), 생성속도(5~7분)
    - fast: 울트라보다 느리게 생성되나 음성 품질 개선됨. 하지만 속도나 톤이 이상해질 수 있음, 생성속도(15~17분)
    - high_quality: 음질 최상, 화자에 따라 속도나 톤이 바뀔 수 있음(울트라 마음에 들어서 돌렸다가 낭패볼 수 있음 주의), 생성속도(25분 정도)
4. output_path: 저장할 음성파일 이름 설정

In [ ]:
import os
import numpy as np
import torch
from tortoise.api import TextToSpeech
from scipy.io.wavfile import write

# CPU에서 사용할 스레드 수 설정
torch.set_num_threads(15)  # 자신의 CPU 코어 수에 맞게 설정하세요.

# 저장된 임베딩 로드 및 Tensor로 변환
embedding_path = "tortoise/conditioning_latents.npz"
loaded_latents = np.load(embedding_path)
conditioning_latents = [torch.tensor(loaded_latents[key]) for key in loaded_latents]

# Tortoise TTS 모델 초기화
tts = TextToSpeech(use_deepspeed=True, kv_cache=True, half=True)

# 텍스트 파일이 있는 폴더 및 결과 저장 폴더 설정
text_folder = "tortoise/text_new/middle_3_02"
output_folder = "tortoise/result/pinocchio"
os.makedirs(output_folder, exist_ok=True)

# 각 텍스트 파일을 한 줄씩 읽어 음성 변환 및 저장
for i in range(1, 11):
    text_file_path = os.path.join(text_folder, f"text_line_{i}.txt")

    try:
        with open(text_file_path, "r", encoding="utf-8") as f:
            text = f.read().strip()  # 파일에서 텍스트를 읽어옴
            if not text:
                continue

            # 텍스트를 화자 샘플의 스타일로 음성 변환
            pcm_audio = tts.tts_with_preset(
                text, conditioning_latents=conditioning_latents, preset="ultra_fast"
            )

            # PCM 데이터를 numpy 배열로 변환 후 최종 WAV 파일로 저장
            output_path = os.path.join(output_folder, f"tortoise_ultra_fast1{i}.wav")
            write(output_path, 22050, np.array(pcm_audio, dtype=np.float32))
            print(f"'{text}' 문장이 '{output_path}'로 저장되었습니다.")

    except FileNotFoundError:
        print(f"텍스트 파일 {text_file_path}을 찾을 수 없습니다.")
    except RuntimeError as e:
        print(f"텍스트 파일 {text_file_path} 처리 중 오류 발생: {e}")

print("모든 텍스트 파일에 대한 음성 변환이 완료되었습니다.")

c:\Users\user\anaconda3\envs\tortoise\lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\user\anaconda3\envs\tortoise\lib\site-packages\transformers\modeling_utils.py:460: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted

Generating autoregressive samples..


100%|██████████| 16/16 [01:11<00:00,  4.47s/it]


Computing best candidates using CLVP


100%|██████████| 16/16 [00:10<00:00,  1.58it/s]


...

'Yes, we have several puppies and older dogs.' 문장이 'tortoise/result/pinocchio\tortoise_ultra_fast12.wav'로 저장되었습니다.
Generating autoregressive samples..


100%|██████████| 16/16 [00:41<00:00,  2.57s/it]


Computing best candidates using CLVP


 56%|█████▋    | 9/16 [00:05<00:04,  1.66it/s]


KeyboardInterrupt: 